In [1]:
from collections import namedtuple, defaultdict
from itertools import groupby, pairwise
from typing import NamedTuple

In [2]:
class Map:
    def __init__(self, data):
        self.height = len(data)
        self.width = len(data[0])
        self.data = [list(row) for row in data]

    def __getitem__(self, p):
        return self.data[p.row][p.col]

    def get_neighbors(self, p):
        if p.row > 0:
            yield p + Point(-1, 0)
        if p.row < self.height - 1:
            yield p + Point(1, 0)
        if p.col > 0 :
            yield p + Point(0, -1)
        if p.col < self.width - 1:
            yield p + Point(0, 1)

    def rows(self):
        for row, line in enumerate(self.data):
             yield [Point(row, col)  for col, item in enumerate(line)]
        
    def points(self):
        yield from (Point(row, col) for row, line in enumerate(self.data)
                    for col, item in enumerate(line))    
            
class Point(NamedTuple):
    row: int
    col: int

    def __add__(self, other):
        return Point(self.row + other.row, self.col + other.col)

In [3]:
def fill(map, start):  
    stack = [(start, set([start]))]
    seen = set([start])

    while stack:
        current, current_group = stack.pop()
        current_crop = map[current]
    
        for bordering in map.get_neighbors(current):
            if bordering in seen:
                continue
            crop = map[bordering]
            if crop == current_crop:
                current_group.add(bordering)
                seen.add(bordering)
                stack.append((bordering, current_group))
    return (seen, map[start])

def permiter(group, crop, map):
    # start with 4 to account for map borders
    # subtract any that are the same crop type
    total_permiter = 0
    for p in group:
        borders = 4
        for bordering in map.get_neighbors(p):
            if map[bordering] == crop:
                borders -= 1
        total_permiter += borders
    return total_permiter

def part_one(map):
    groups = []
    seen = set()
    
    for p in map.points():
        if p in seen:
            continue
        group, crop = fill(map, p)
        groups.append((group, crop))
        seen.update(group)
        
    return groups

In [4]:
sample_input = '''RRRRIICCFF
RRRRIICCCF
VVRRRCCFFF
VVRCCCJFFF
VVVVCJJCFE
VVIVCCJJEE
VVIIICJJEE
MIIIIIJJEE
MIIISIJEEE
MMMISSJEEE'''.split('\n')

sample_map = Map(sample_input)

groups = part_one(sample_map)
sum((len(g) * permiter(g, crop, sample_map))  for g, crop in groups)

1930

In [5]:
with open('input_files/12.txt') as f:
    big_map = Map(f.read().splitlines())

groups = part_one(big_map)
sum((len(g) * permiter(g, crop, big_map))  for g, crop in groups)

1473620

## Part Two

Stand on each point in the group and look around to see if you at a corner.

In [6]:
def count_corners(p, map, points_to_group):
    
    group = points_to_group[p]
    
    side_count =  sum(1 for n in [Point(p[0], p[1]+1), Point(p[0], p[1]-1)] if n not in group)
    corners = 0

    # look down
    if Point(p[0]+1, p[1]) not in group:
        corners += side_count
    else:
        if Point(p[0], p[1]-1) in group and Point(p[0]+1, p[1]-1) not in group:
            corners += 1
        if Point(p[0], p[1]+1) in group and Point(p[0]+1, p[1]+1) not in group:
            corners += 1        
    
    # look up            
    if Point(p[0]-1, p[1]) not in group:
        corners += side_count
    else:
        if Point(p[0], p[1]-1) in group and Point(p[0]-1, p[1]-1) not in group:
            corners += 1
        if Point(p[0], p[1]+1) in group and Point(p[0]-1, p[1]+1) not in group:
            corners += 1
    
    return corners

def part_two(map):
    groups = part_one(map)
    
    points_to_group = {}

    for g, crop in groups:
        fg = frozenset(g)
        for p in g:
            points_to_group[p] = fg

    total = 0

    for g, crop in groups:
        group_count = 0
        for p in g:
            group_count += count_corners(p, map, points_to_group)
        total += group_count * len(g)
        
    
    return total

In [7]:
sample_input = '''RRRRIICCFF
RRRRIICCCF
VVRRRCCFFF
VVRCCCJFFF
VVVVCJJCFE
VVIVCCJJEE
VVIIICJJEE
MIIIIIJJEE
MIIISIJEEE
MMMISSJEEE'''.split('\n')

sample_map = Map(sample_input)

part_two(sample_map)

1206

In [8]:
with open('input_files/12.txt') as f:
    big_map = Map(f.read().splitlines())
part_two(big_map)

902620